### eval mertric
http://posco-aichallenge.kr/project_02.html

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./swell_data_by_hour.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34992 entries, 0 to 34991
Data columns (total 4 columns):
hour                     34992 non-null int64
climate                  34992 non-null object
weather_deterioration    34992 non-null int64
swell                    34392 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.1+ MB


In [3]:
df.set_index('hour',inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34992 entries, 2014010407 to 2018010106
Data columns (total 3 columns):
climate                  34992 non-null object
weather_deterioration    34992 non-null int64
swell                    34392 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.1+ MB


In [19]:
df['pred'] = np.random.choice(2,len(df))

In [20]:
df

,climate,weather_deterioration,swell,pred
hour,,,,
2014010407,0,0,0.0,0
2014010408,0,0,0.0,1
2014010409,0,0,0.0,0
2014010410,0,0,0.0,1
2014010411,0,0,0.0,1
2014010412,0,0,0.0,0
2014010413,0,0,0.0,1
2014010414,0,0,0.0,0
2014010415,-1.0,1,0.0,1


In [87]:
class evaluation:
    def __init__(self, true_s):
        self.true_df = pd.DataFrame(true_s)
        self.true_df.columns = ['swell']
        self.true_df['swell(t+1)'] = self.true_df['swell'].shift(-1)
        self.true_df['swell(t+2)'] = self.true_df['swell'].shift(-2)
        self.true_df['swell(t-1)'] = self.true_df['swell'].shift(1)
        self.true_df['swell(t-2)'] = self.true_df['swell'].shift(2)
    
        self.swell_columns = ['swell(t+1)','swell(t+2)','swell(t-1)','swell(t-2)']
    
    def _eval_one(self, pred):
        true = self.true_df.loc[pred.name]
        swell = true['swell'] 
        p_swell = pred['pred']

        if swell == 0 and p_swell == 0:
            return 1

        if swell == 1 and p_swell == 1:
            return 2

        if swell == 0 and p_swell == 1:
            if (true[self.swell_columns].values == 1).any():
                return 0
            else:
                return -1

        if swell == 1 and p_swell == 0:
            if (pred[self.swell_columns].values == 1).any():
                return 0
            else: 
                return -2
    
    def evaluate(self, pred):
        pred_df = pd.DataFrame(pred)
        pred_df.columns = ['pred']
        pred_df['swell(t+1)'] = pred_df['pred'].shift(-1)
        pred_df['swell(t+2)'] = pred_df['pred'].shift(-2)
        pred_df['swell(t-1)'] = pred_df['pred'].shift(1)
        pred_df['swell(t-2)'] = pred_df['pred'].shift(2)

        result = []
        for index in pred_df.index:
            result.append(self._eval_one(pred_df.loc[index]))
        
        print(result)
        
        return np.sum(result)

In [88]:
eval_metric = evaluation(df['swell'])

In [89]:
eval_metric.evaluate(df['pred'].iloc[:5])

[1, -1, 1, -1, -1]


-1

In [90]:
test_df = pd.DataFrame([0,0,1,1,1,1,1,0],columns=['swell'])
test_df['pred1'] = [0,0,1,1,1,1,1,0]
test_df['pred2'] = [0,1,1,1,0,0,0,0]
test_df['pred3'] = [0,0,0,0,0,0,0,0]
test_df['pred4'] = [1,1,1,1,1,1,1,1]
test_df['pred5'] = [1,1,0,0,0,0,0,1]

In [91]:
test_df

,swell,pred1,pred2,pred3,pred4,pred5
0,0,0,0,0,1,1
1,0,0,1,0,1,1
2,1,1,1,0,1,0
3,1,1,1,0,1,0
4,1,1,0,0,1,0
5,1,1,0,0,1,0
6,1,1,0,0,1,0
7,0,0,0,0,1,1


In [92]:
eval_metric = evaluation(test_df['swell'])

In [98]:
for col in test_df.columns:
    print(eval_metric.evaluate(test_df[col]))

[1, 1, 2, 2, 2, 2, 2, 1]
13
[1, 1, 2, 2, 2, 2, 2, 1]
13
[1, 0, 2, 2, 0, 0, -2, 1]
4
[1, 1, -2, -2, -2, -2, -2, 1]
-7
[0, 0, 2, 2, 2, 2, 2, 0]
10
[0, 0, 0, 0, -2, 0, 0, 0]
-2
